In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
experiments = ['IEEE9_noiseless_red', 'IEEE9_noisy_red']
architectures = ['fcn', 'inception', 'mcdcnn', 'tlenet']

## k-fold

In [4]:
df_experiments = dict.fromkeys(experiments)

for experiment in experiments:
    df_res_arch = dict.fromkeys(architectures)
    for architecture in architectures:
        path_res = os.path.join(os.getcwd(), 'experiments', experiment, architecture, 'k-fold')
        print(path_res)
        epoch_best = 0
        for n in range(1, 6):
            path_best = os.path.join(path_res, f'{n}_df_best_model.csv')
            df_path_best = pd.read_csv(path_best)
            epoch_best += df_path_best['best_model_nb_epoch'].values[0]
        epoch_best /= n
        epoch_best = int(epoch_best)
        
        path_res = os.path.join(os.getcwd(), 'experiments', experiment, architecture, 'k-fold', 'results_k-fold.csv')
        df_res = pd.read_csv(path_res)
        
        f1_mean = df_res['F1'].mean() * 100
        f1_std = df_res['F1'].std() * 100
        
        acc_mean = df_res['accuracy'].mean() * 100
        acc_std = df_res['accuracy'].std() * 100
        
        prec_mean = df_res['precision'].mean() * 100
        prec_std = df_res['precision'].std() * 100
        
        rec_mean = df_res['recall'].mean() * 100
        rec_std = df_res['recall'].std() * 100
        
        pred_t_mean = df_res['prediction_time'].mean()
        pred_t_std = df_res['prediction_time'].std()
        
        df_res_arch[architecture] = {'f1': f'{f1_mean:.4f}+-{f1_std:.4f}', 
         'accuracy' : f'{acc_mean:.4f}+-{acc_std:.4f}', 
         'precision' :  f'{prec_mean:.4f}+-{prec_std:.4f}', 
         'recall' : f'{rec_mean:.4f}+-{rec_std:.4f}', 
         'prediction_time' : f'{pred_t_mean:.4f} +- {pred_t_std:.4f}', 
        'epoch_best' : f'{epoch_best}'}
        
        df_res = None
    
    df_experiments[experiment] = df_res_arch
    df_res_arch = None

/Users/sergio.dorado/Documents/GitHub_Repositories/01_Maintained/Papers/dl_tsc_small_signal/experiments/IEEE9_noiseless_red/fcn/k-fold
/Users/sergio.dorado/Documents/GitHub_Repositories/01_Maintained/Papers/dl_tsc_small_signal/experiments/IEEE9_noiseless_red/inception/k-fold
/Users/sergio.dorado/Documents/GitHub_Repositories/01_Maintained/Papers/dl_tsc_small_signal/experiments/IEEE9_noiseless_red/mcdcnn/k-fold
/Users/sergio.dorado/Documents/GitHub_Repositories/01_Maintained/Papers/dl_tsc_small_signal/experiments/IEEE9_noiseless_red/tlenet/k-fold
/Users/sergio.dorado/Documents/GitHub_Repositories/01_Maintained/Papers/dl_tsc_small_signal/experiments/IEEE9_noisy_red/fcn/k-fold
/Users/sergio.dorado/Documents/GitHub_Repositories/01_Maintained/Papers/dl_tsc_small_signal/experiments/IEEE9_noisy_red/inception/k-fold
/Users/sergio.dorado/Documents/GitHub_Repositories/01_Maintained/Papers/dl_tsc_small_signal/experiments/IEEE9_noisy_red/mcdcnn/k-fold
/Users/sergio.dorado/Documents/GitHub_Reposito

In [5]:
df_experiments

{'IEEE9_noiseless_red': {'fcn': {'f1': '86.8398+-5.9240',
   'accuracy': '87.5000+-5.8926',
   'precision': '86.4470+-5.1666',
   'recall': '90.8088+-4.3656',
   'prediction_time': '0.2146 +- 0.0007',
   'epoch_best': '8'},
  'inception': {'f1': '97.2676+-6.1099',
   'accuracy': '97.5000+-5.5902',
   'precision': '97.0000+-6.7082',
   'recall': '98.2353+-3.9460',
   'prediction_time': '1.1645 +- 0.0934',
   'epoch_best': '19'},
  'mcdcnn': {'f1': '100.0000+-0.0000',
   'accuracy': '100.0000+-0.0000',
   'precision': '100.0000+-0.0000',
   'recall': '100.0000+-0.0000',
   'prediction_time': '0.2037 +- 0.0469',
   'epoch_best': '59'},
  'tlenet': {'f1': '40.5854+-0.8015',
   'accuracy': '68.3333+-2.2822',
   'precision': '34.1667+-1.1411',
   'recall': '50.0000+-0.0000',
   'prediction_time': '0.8920 +- 0.0921',
   'epoch_best': '433'}},
 'IEEE9_noisy_red': {'fcn': {'f1': '69.9653+-12.3973',
   'accuracy': '74.1667+-8.0147',
   'precision': '82.8941+-2.8339',
   'recall': '72.3007+-10.48

435

In [34]:
df_best_epoch

{'cnn': 435,
 'fcn': 274,
 'inception': 76,
 'mcdcnn': 119,
 'mlp': 4270,
 'tlenet': 426}

In [42]:
df_res

,F1,accuracy,recall,duration,precision,prediction_time
0,0.996572,0.996923,0.997674,513.097331,0.995495,0.138017
1,0.996572,0.996923,0.997674,137.838727,0.995495,0.134646
2,1.000000,1.000000,1.000000,99.610077,1.000000,0.135558
3,1.000000,1.000000,1.000000,97.351765,1.000000,0.133909
4,1.000000,1.000000,1.000000,97.022870,1.000000,0.136471
